# Simple SPX weekly range calculation tool

this code is intended as an example of gathering market data and providing some estimation of a weekly move without access to the instruments option chain. 
there are many ways to determine exepected move, the most straight forward is to take an at the money straddle with the desired expiration date (0 days to expiration (DTE), 1, 5, etc).
if one does not have options api access another way is to use the current vix on market open.
an obvious assumption then is that market volatility is accurately represented by vix, which is not always true, we can be 'under-vixed' or 'over-vixed'. 
the approach outlined below was reasonable in 2021 and many prior years.
in 2022, vix has been a poor indicator of intrinsic value, where one desires intrinsic value (IV) > realized value (RV).
effectively if the vix is not accurately pricing volatility, this method is *less* reliable.
nevertheless, the example is instructive, as the goal of this example is not to provide a discussion on volatility modeling but provide a rough order of magnitude approximation of expected move lacking other, more detailed information.

here i will re-iterate that this is not intended to use for financial decisions.
your financial decisions are your own responsibility.

In [1]:
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# we access publicly available data for the current year
# ytd representing all market data from Jan 1
# we acquire weekly data and assemble a data frame

stocks = yf.download(
    tickers='^GSPC ^VIX',
    # use "period" instead of start/end
    # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    # (optional, default is '1mo')
    period = "ytd",
    # fetch data by interval (including intraday if period < 60 days)
    # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    # (optional, default is '1d')
    interval = "1wk",
    # group by ticker (to access via data['SPY'])
    # (optional, default is 'column')
    group_by = 'ticker',
    # adjust all OHLC automatically
    # (optional, default is False)
    auto_adjust = True,
    # download pre/post regular market hours data
    # (optional, default is False)
    prepost = True,
    # use threads for mass downloading? (True/False/Integer)
    # (optional, default is True)
    threads = True
)
stocks = stocks.dropna()

[*********************100%***********************]  2 of 2 completed


## The weekly range calculation and considerations
1. first we scale weekly SPX open by VIX/100 * sqrt(5/252)<br>
    1.1 the intent is for a weekly iron condor opened early monday to expire friday<br>
    1.2    therefore, the numerator (DTE) is 5<br>
    1.3    we normalize to the number of approximate trading days in a year, 252<br>
2. we assume a normal distribution for call and put<br>
    2.1    we scale the vix_price to 1.3 to achieve a higher probability of profit (otherwise 1 EM is ~68% win rate by theory)<br>
    2.2   i've included the ability to add skew (which is what i really do) on the upper/lower scalar to move up and down a presumed standard distribution. <br>
    2.3    if you the data from the options chain to determine call/put balanced risk you're not really needing to use VIX to determine expected move.<br>

In [3]:
def calc_weekly_range(df, lower_scalar=1.3, upper_scalar=1.3):
    # primary utility function for determining weekly ranges
    df['vix_price'] = df['^GSPC', 'Open'] * df['^VIX', 'Open']/100 * np.sqrt(5) / np.sqrt(252)
    # this is rounded to the nearest 5 as SPX options are in increments of 5
    df['vix_low'] =  5 * round((df['^GSPC', 'Open'] - df.vix_price * lower_scalar)/5)
    df['vix_high'] = 5 * round((df['^GSPC', 'Open'] + df.vix_price * upper_scalar)/5)
    # bracketing outcomes (only valid at *end of week*) to count historical win/loss/risk rate
    df['win'] = (df['^GSPC', 'Close'] >  df['vix_low']) & (df['^GSPC', 'Close'] <  df['vix_high'])
    # not all breaches result in a loss so we count the winning/losing/at risk sides
    # in the future this can be used to show a *lack* of symmetry in the previously assumed normal distribution
    df['breach'] = (df['^GSPC', 'Low'] <  df['vix_low']) | (df['^GSPC', 'High'] >  df['vix_high'])
    df['breach_low'] = df['^GSPC', 'Low'] <  df['vix_low']
    df['breach_high'] = df['^GSPC', 'High'] >  df['vix_high']
    return df

In [4]:
# execute utility function and augment dataframe
df = calc_weekly_range(stocks)

## Quick Stats on historical outcomes

In [5]:
# count wins
df.win.value_counts('True')

True     0.84
False    0.16
Name: win, dtype: float64

In [6]:
# count frequency of RV>IV (over some time frame)
# note True here does not exactly account for losses above
df.breach.value_counts('True')

False    0.8
True     0.2
Name: breach, dtype: float64

In [7]:
# breach frequency on the put side
df.breach_low.value_counts('True')

False    0.92
True     0.08
Name: breach_low, dtype: float64

In [8]:
# breach frequency on the call side
df.breach_high.value_counts('True')

False    0.88
True     0.12
Name: breach_high, dtype: float64

## Quick inspection on last and current weeks

In [9]:
# last weeks results for inspection
df.iloc[-2]

^GSPC        Open       4052.02002
             High      4052.449951
             Low       3918.389893
             Close     3934.379883
             Volume    20662410000
^VIX         Open        20.299999
             High        23.280001
             Low         19.780001
             Close           22.83
             Volume              0
vix_price               115.865019
vix_low                     3900.0
vix_high                    4180.0
win                           True
breach                       False
breach_low                   False
breach_high                  False
Name: 2022-12-05 00:00:00, dtype: object

In [10]:
# this weeks projections / performance to date
df.iloc[-1]

^GSPC        Open      3954.169922
             High       3977.02002
             Low       3933.040039
             Close     3934.379883
             Volume     2123231000
^VIX         Open        22.549999
             High        23.209999
             Low             22.18
             Close           22.83
             Volume              0
vix_price               125.599119
vix_low                     3790.0
vix_high                    4090.0
win                           True
breach                       False
breach_low                   False
breach_high                  False
Name: 2022-12-09 00:00:00, dtype: object

## Saving data

In [11]:
# saved the table in human readable format
df.to_excel('spx_range_backtest.xlsx')

In [12]:
# just visual inspection to check for accuracy 
# there are other interesting things here
stocks

^GSPC                                                      \
                   Open         High          Low        Close       Volume   
Date                                                                          
2022-01-03  4778.140137  4818.620117  4662.740234  4677.029785  21878940000   
2022-01-10  4655.339844  4748.830078  4582.240234  4662.850098  21251840000   
2022-01-17  4632.240234  4632.240234  4395.339844  4397.939941  19444410000   
2022-01-24  4356.319824  4453.229980  4222.620117  4431.850098  27889090000   
2022-01-31  4431.790039  4595.310059  4414.020020  4500.529785  23505280000   
2022-02-07  4505.750000  4590.029785  4401.410156  4418.640137  23778810000   
2022-02-14  4412.609863  4489.549805  4327.220215  4348.870117  22562340000   
2022-02-21  4332.740234  4385.339844  4114.649902  4384.649902  21848520000   
2022-02-28  4354.169922  4416.779785  4279.540039  4328.870117  28092740000   
2022-03-07  4327.009766  4327.009766  4157.870117  4204.310059  29765730000   
2022-03-14  4202.750000  4465.399902  4161.720215  4463.120117  30540660000   
2022-03-21  4462.399902  4546.029785  4424.299805  4543.060059  22611740000   
2022-03-28  4541.089844  4637.299805  4507.569824  4545.859863  23169700000   
2022-04-04  4547.970215  4593.450195  4450.040039  4488.279785  23239490000   
2022-04-11  4462.640137  4471.000000  4381.339844  4392.589844  16329620000   
2022-04-18  4385.629883  4512.939941  4267.620117  4271.779785  21597890000   
2022-04-25  4255.339844  4308.450195  4124.279785  4131.930176  24637900000   
2022-05-02  4130.609863  4307.660156  4062.510010  4123.339844  25076170000   
2022-05-09  4081.270020  4081.270020  3858.870117  4023.889893  29126270000   
2022-05-16  4013.020020  4090.719971  3810.320068  3901.360107  24603940000   
2022-05-23  3919.419922  4158.490234  3875.129883  4158.240234  23231370000   
2022-05-30  4151.089844  4177.509766  4073.850098  4108.540039  19471340000   
2022-06-06  4134.720215  4168.779785  3900.159912  3900.860107  21764190000   
2022-06-13  3838.149902  3838.149902  3636.870117  3674.840088  30486930000   
2022-06-20  3715.310059  3913.649902  3715.310059  3911.739990  23570150000   
2022-06-27  3920.760010  3945.860107  3738.669922  3825.330078  21693690000   
2022-07-04  3792.610107  3918.500000  3742.060059  3899.379883  17073700000   
2022-07-11  3880.939941  3880.939941  3721.560059  3863.159912  19693570000   
2022-07-18  3883.790039  4012.439941  3818.629883  3961.629883  20385270000   
2022-07-25  3965.719971  4140.149902  3910.739990  4130.290039  20488830000   
2022-08-01  4112.379883  4167.660156  4079.810059  4145.189941  21651540000   
2022-08-08  4155.930176  4280.470215  4112.089844  4280.149902  21098400000   
2022-08-15  4269.370117  4325.279785  4218.700195  4228.479980  19013350000   
2022-08-22  4195.080078  4203.040039  4057.659912  4057.659912  18714200000   
2022-08-29  4034.580078  4062.989990  3903.649902  3924.260010  19592960000   
2022-09-05  3930.889893  4076.810059  3886.750000  4067.360107  15886450000   
2022-09-12  4083.669922  4119.279785  3837.080078  3873.330078  24728470000   
2022-09-19  3849.909912  3907.070068  3647.469971  3693.229980  21332100000   
2022-09-26  3682.719971  3736.739990  3584.129883  3585.620117  24475900000   
2022-10-03  3609.780029  3806.909912  3604.929932  3639.659912  22948200000   
2022-10-10  3647.510010  3712.000000  3491.580078  3583.070068  21864890000   
2022-10-17  3638.649902  3762.790039  3638.649902  3752.750000  22634960000   
2022-10-24  3762.010010  3905.419922  3741.649902  3901.060059  23555090000   
2022-10-31  3881.850098  3911.790039  3698.149902  3770.550049  24226300000   
2022-11-07  3780.709961  4001.479980  3744.219971  3992.929932  24968840000   
2022-11-14  3977.969971  4028.840088  3906.540039  3965.340088  21831700000   
2022-11-21  3956.229980  4034.020020  3933.340088  4026.120117  12724860000   
2022-11-28  4005.360107  4100.509766  3937.649902  4071.699951  2228